# Without RAG

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
model=ChatOpenAI()

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","your are worlt class technical documentation writer."),
        ("user","{input}")
    ]
)

In [4]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()

In [5]:
chain=prompt | model | output_parser

In [30]:
print(chain.invoke(
      {
          "input":"how is europe vote result 2024 in Germany?"
      }))

I'm sorry, but as of my last update, there is no such thing as "gpt4o." It is possible that it is a typo or a new term that has emerged since my last update. If you can provide more context or information, I may be able to help you further.


# RAG

In [24]:
#data loader
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://en.wikipedia.org/wiki/2024_European_Parliament_election_in_Germany")
docs=loader.load()

In [25]:
docs

[Document(page_content='\n\n\n\n2024 European Parliament election in Germany - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBackground\n\n\n\n\n\n\n\n\n2\nElectoral threshold\n\n\n\n\n\n\n\n\n3\nOutgoing delegation\n\n\n\n\n\n\n\n\n4\nParties campaign

In [26]:
# embedding
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [27]:
from langchain_community.vectorstores import FAISS

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter()
documents=text_splitter.split_documents(docs)

In [29]:
documents

[Document(page_content='2024 European Parliament election in Germany - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBackground\n\n\n\n\n\n\n\n\n2\nElectoral threshold\n\n\n\n\n\n\n\n\n3\nOutgoing delegation\n\n\n\n\n\n\n\n\n4\nParties campaigning for 

In [31]:
vector=FAISS.from_documents(documents,embeddings)

In [32]:
# create chain
from langchain_core.prompts import ChatPromptTemplate

In [33]:
prompt =ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context
<context>
{context}
</context>
Question: {input}
"""
)

In [36]:
# creating chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(model,prompt)

In [37]:
from langchain.chains import create_retrieval_chain
retriever=vector.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [38]:
response=retrieval_chain.invoke(
    {
        "input":"how is europe vote result 2024 in Germany?"
    }
)

In [39]:
response

{'input': 'how is europe vote result 2024 in Germany?',
 'context': [Document(page_content='2024 European Parliament election in Germany - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBackground\n\n\n\n\n\n\n\n\n2\nElectoral threshold\n\n\n\n\n\n\n\n\

In [41]:
print(response["answer"])

The European Parliament election result in Germany in 2024 showed the following distribution of seats:

- CDU/CSU: 29 seats
- AfD: 15 seats
- SPD: 14 seats
- Greens: 12 seats
- BSW: 6 seats
- FDP: 5 seats
- Left: 3 seats
- FW: 3 seats
- Volt: 3 seats
- Die PARTEI: 2 seats
- Tierschutzpartei: 1 seat
- ÖDP: 1 seat
- Familie: 1 seat
- PDF: 1 seat
